In [ ]:
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv(dotenv_path=".env", override=True)

from utils.auth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentEvaluationRequest,
    InputDataset,
    EvaluatorIds,
    EvaluatorConfiguration,
    AgentEvaluationSamplingConfiguration,
    AgentEvaluationRedactionConfiguration,
)

# not using the settings.agent_name, since this agent name is used by other notebooks
AGENT_NAME = "my-eval-agent-1"
AGENT_INSTRUCTIONS = "You are helpful agent"

project_client = AIProjectClient(
    endpoint=settings.project_endpoint, 
    credential=credential, 
#   api_version=settings.project_api_version
)

# [START evaluations_agent_sample]
agent = project_client.agents.create_agent(
    model=settings.model_deployment_name,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
)
print(f"Created agent, agent ID: {agent.id}")

thread = project_client.agents.threads.create()
print(f"Created thread, thread ID: {thread.id}")

message = project_client.agents.messages.create(
    thread_id=thread.id, role="user", content="Hello, tell me a joke"
)
print(f"Created message, message ID: {message.id}")

run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

Created agent, agent ID: asst_yDQOgHuLYD5zhwoebWPPunL7
Created thread, thread ID: thread_vy65J3wNJgTOPTfs1cTauEqr
Created message, message ID: msg_CLdpk67ZhwE0zX2MQjXVUODW
Run status: RunStatus.COMPLETED


In [9]:
agent_evaluation_request = AgentEvaluationRequest(
    run_id=run.id,
    thread_id=thread.id,
    evaluators={
        "violence": EvaluatorConfiguration(
            id=EvaluatorIds.VIOLENCE,
        )
    },
    sampling_configuration=AgentEvaluationSamplingConfiguration(
        name="test",
        sampling_percent=100,
        max_request_rate=100,
    ),
    redaction_configuration=AgentEvaluationRedactionConfiguration(
        redact_score_properties=False,
    ),
    app_insights_connection_string=project_client.telemetry.get_connection_string(),
)

agent_evaluation_response = project_client.evaluations.create_agent_evaluation(
    evaluation=agent_evaluation_request
)

print(agent_evaluation_response)

{'id': 'thread_vy65J3wNJgTOPTfs1cTauEqr;run_Pt8pHZe7g96JBV89nWwuKKcf', 'status': 'Running', 'result': None, 'error': None}


In [10]:
project_client.close()